In [8]:
from Train import train
from DynamicsModel import model as DynamicsModel
from env1.env1 import InvertedPendulum as env1
from env2.env2 import InvertedPendulum as env2
import jax 
import jax.numpy as jnp
from sklearn.model_selection import train_test_split


In [2]:
# Load Data:
X_data1 = jnp.load('Data/statest.npy')
X_data2 = jnp.load('Data/actions.npy')

X_data = jnp.concatenate((X_data1, X_data2), axis=1)
y_data = jnp.load('Data/statestp.npy')
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=0)

rng = jax.random.PRNGKey(0)

In [3]:
params = train(rng, X_train, y_train, X_test, y_test, observation_size = 4,action_size = 1, learning_rate=0.001, num_epochs=100, batch_size=64)

Epoch 0, Train Loss: 76.6121, Test Loss: 68.3532
Epoch 10, Train Loss: 4.5359, Test Loss: 1.5068
Epoch 20, Train Loss: 1.0776, Test Loss: 1.3087
Epoch 30, Train Loss: 0.7698, Test Loss: 1.1606
Epoch 40, Train Loss: 0.5982, Test Loss: 1.0225
Epoch 50, Train Loss: 0.4162, Test Loss: 0.7942
Epoch 60, Train Loss: 0.4019, Test Loss: 0.6866
Epoch 70, Train Loss: 0.3219, Test Loss: 0.5658
Epoch 80, Train Loss: 0.2734, Test Loss: 0.4934
Epoch 90, Train Loss: 0.2649, Test Loss: 0.5164


In [9]:
env1 = env1()
key = jax.random.key(0)

State = env1.reset(key)
State1 = env1.step(State, 1.0)

qqd = jnp.concatenate((jnp.expand_dims(State.pipeline_state.q,axis=1),jnp.expand_dims(State.pipeline_state.qd, axis=1)), axis=0)[:,0]
qqd10 = jnp.concatenate((jnp.expand_dims(State1.pipeline_state.q,axis=1),jnp.expand_dims(State1.pipeline_state.qd, axis=1)), axis=0)

model = DynamicsModel(4,5)
input = jnp.concatenate((qqd, jnp.array([1.0])), axis=0)
qqd11 = model.apply({"params":params}, input )

print(qqd10,qqd11)


11
[[-4.364456 ]
 [-3.306514 ]
 [ 2.0459676]
 [-0.8970223]] [-4.0854964 -3.108387   1.1106935 -1.363312 ]
